In [ ]:
import os
import pandas as pd
import math
import matplotlib as plt

In [ ]:
def readStatelog(folder, file):
    return pd.read_csv(os.path.join(folder,file), skiprows = 1)

def readGeneralLog(folder, file):
    conflicts_ref = pd.read_csv(os.path.join(folder,file), skiprows = 3)
    conflicts_ref = conflicts_ref.drop([0])
    return conflicts_ref

def readFolder(folder):
    files_dict = {}
    for file in os.listdir(folder):
        if file.split('_')[0] == 'STATELOG':
            statelog = readStatelog(folder, file)
            files_dict['USEPESTATELOG'] = statelog
        else:
            files_dict[file.split('_')[0]] = readGeneralLog(folder, file)
    
    return files_dict

def distnace(lat1,lon1,lat2,lon2):
    lat1 = lat1 * math.pi/180
    lat2 = lat2 * math.pi/180
    lon1 = lon1 * math.pi/180
    lon2 = lon2 * math.pi/180
    
    return math.acos(math.sin(lat1)*math.sin(lat2)+math.cos(lat1)*math.cos(lat2)*math.cos(lon2-lon1))*6371

def readLogs(folder):
    files_dict = readFolder(folder)
        
    conflog = files_dict['USEPECONFLOG']
    cpalog = files_dict['USEPECPALOG']
    flightlog = files_dict['USEPEFLIGHTLOG']
    loslog = files_dict['USEPELOSLOG']
    statelog = files_dict['USEPESTATELOG']
    
    return statelog, conflog, loslog, cpalog, flightlog

# Analysing flight log

In [ ]:
def analyseFlightLog(flightlog):
    flightlog['# Simulation Time [s]'] = flightlog['# Simulation Time [s]'].astype(float)

    flightlog = flightlog[['SURVEILLANCE_' not in x for x in flightlog[' UAS']]]

    flightlog_first = flightlog.groupby(' UAS').first()
    started_flights = flightlog_first[flightlog_first[' Takeoff/Landing'] == 'takeoff'].index

    flightlog_last = flightlog.groupby(' UAS').last()
    completed_flights = flightlog_last[flightlog_last[' Takeoff/Landing'] == 'landing'].index

    print("Started flights: {}".format(len(started_flights)))
    print("Completed flights: {}".format(len(completed_flights)))
    
    flightlog_landed = flightlog[flightlog[' UAS'].isin(completed_flights)]
    total_distance = sum(flightlog_landed.groupby(' UAS').sum()[' Distance [m]'])
    
    avg_distance_flown = total_distance / len(completed_flights)

    print( "Average distance flown: {}".format(avg_distance_flown ))
    
    df = flightlog[flightlog[' UAS'].isin(completed_flights)].groupby(' Takeoff/Landing').sum()
    total_time = df.loc['landing']['# Simulation Time [s]'] - df.loc['takeoff']['# Simulation Time [s]']
    
    avg_flight_time = total_time / len(completed_flights)

    print( "Average flight time: {}".format(avg_flight_time ))
    
    return len(started_flights), len(completed_flights), avg_distance_flown, avg_flight_time
    

# Analysing conflicts

In [ ]:
def analyseConflicts(conflog, threshold=20):
    conflog['# Simulation Time [s]'] = conflog['# Simulation Time [s]'].astype(float)
    
    conflog_filtered = filterConflict(conflog, threshold)
    
    print("Number of conflicts: {}".format(len(conflog_filtered)))
    print("Average conflict time: {}".format(conflog_filtered['time_in_conflict'].mean()))
    
    return len(conflog_filtered), conflog_filtered['time_in_conflict'].mean(), conflog_filtered

def filterConflict(conflog, threshold):
    conflicts_dict = {}
    conflicts_dict['UAS1'] = []
    conflicts_dict['UAS2'] = []
    conflicts_dict['start'] = []
    conflicts_dict['end'] = []
    conflicts_dict['time_in_conflict'] = []

    for UAS1 in conflog[' UAS1'].unique():
        df = conflog[conflog[' UAS1'] == UAS1]
        for UAS2 in df[' UAS2'].unique():
            df2 = df[df[' UAS2'] == UAS2] 
            start = True
            time_in_conflict = 0
            for idx, row in df2.iterrows():

                if row[' Start/End'] == 'start':
                    if start:
                        start_time = row['# Simulation Time [s]']
                        prev_start_time = row['# Simulation Time [s]']
                        start = False
                    else:
                        if row['# Simulation Time [s]'] > end_time + threshold:
                            conflicts_dict['UAS1'] += [UAS1]
                            conflicts_dict['UAS2'] += [UAS2]
                            conflicts_dict['start'] += [start_time]
                            conflicts_dict['end'] += [end_time]
                            conflicts_dict['time_in_conflict'] += [time_in_conflict]


                            time_in_conflict = 0
                            start_time = row['# Simulation Time [s]']
                            prev_start_time = row['# Simulation Time [s]']

                        else:
                            prev_start_time = row['# Simulation Time [s]'] 
                    completed = False
                else:
                    end_time = row['# Simulation Time [s]']
                    time_in_conflict += row['# Simulation Time [s]'] - prev_start_time
                    completed = True

            if completed:
                conflicts_dict['UAS1'] += [UAS1]
                conflicts_dict['UAS2'] += [UAS2]
                conflicts_dict['start'] += [start_time]
                conflicts_dict['end'] += [end_time]
                conflicts_dict['time_in_conflict'] += [time_in_conflict]



    conflog_filtered = pd.DataFrame.from_dict(conflicts_dict)
    return conflog_filtered

def filterDeliveryPoint(conflog_filtered, statelog, threshold = 0.25):
    df = pd.DataFrame()
    for idx, row in conflog_filtered.iterrows():
        if ('DELIVERY' in row['UAS1']) or ('DELIVERY' in row['UAS2']):
            #time = math.ceil((row['end'] + row['start'])/2)
            time = row['start']        

            uas1 = statelog[(statelog['# simt'] == time) & (statelog[' id'] == row['UAS1'] ) ].iloc[0]
            uas2 = statelog[(statelog['# simt'] == time) & (statelog[' id'] == row['UAS2'] ) ].iloc[0]

            lat_ref = 52.3594
            lon_ref = 9.7499

            lat1 = uas1[' lat']
            lon1 = uas1[' lon']
            lat2 = uas2[' lat']
            lon2 = uas2[' lon']

            d1 = distnace(lat_ref,lon_ref,lat1,lon1)
            d2 = distnace(lat_ref,lon_ref,lat2,lon2)

            if (d1 < threshold) or (d2 < threshold):
                delivery_region = True
            else:
                delivery_region = False
                df = pd.concat([df, pd.DataFrame(row).T]).reset_index(drop=True)
            
    return df
        

# Analysing LoS

In [ ]:
def analyseLoS(loslog, threshold=20):
    loslog['# Simulation Time [s]'] = loslog['# Simulation Time [s]'].astype(float)
    
    loslog_filtered = filterConflict(loslog, threshold)
    
    print("Number of LoS: {}".format(len(loslog_filtered)))
    print("Average LoS time: {}".format(loslog_filtered['time_in_conflict'].mean()))

    return len(loslog_filtered), loslog_filtered['time_in_conflict'].mean(), loslog_filtered

# Analysing CPA

In [ ]:
def analyseCpa(cpalog, threshold=20):
    cpalog['# Simulation Time [s]'] = cpalog['# Simulation Time [s]'].astype(float)
    cpalog_filtered = filterCpa(cpalog, threshold)
    
    NMAC = cpalog_filtered[cpalog_filtered['distance'] < 3.75].copy().reset_index(drop=True)
    
    print("Number of NMACs: {}".format(len(NMAC)))
    
    return len(NMAC), NMAC
    
def filterCpa(cpalog, threshold):
    conflicts_dict = {}
    conflicts_dict['UAS1'] = []
    conflicts_dict['UAS2'] = []
    conflicts_dict['distance'] = []
    conflicts_dict['time'] = []

    for UAS1 in cpalog[' UAS1'].unique():
        df = cpalog[cpalog[' UAS1'] == UAS1]
        for UAS2 in df[' UAS2'].unique():
            df2 = df[df[' UAS2'] == UAS2]
            
            start = True
            distance = 99
            for idx, row in df2.iterrows():
                if start:
                    start_time = row['# Simulation Time [s]']
                    distance = row[' Distance [m]']
                    time = row['# Simulation Time [s]']
                    start = False
                else:
                    if row['# Simulation Time [s]'] > start_time + threshold:
                        conflicts_dict['UAS1'] += [UAS1]
                        conflicts_dict['UAS2'] += [UAS2]
                        conflicts_dict['distance'] += [distance]
                        conflicts_dict['time'] += [time]

                        start_time = row['# Simulation Time [s]']
                        distance = row[' Distance [m]']
                        time = row['# Simulation Time [s]']

                    else:
                        if row[' Distance [m]'] < distance:
                            distance = row[' Distance [m]']
                            time = row['# Simulation Time [s]']
                        start_time = row['# Simulation Time [s]']
                            
                        prev_start_time = row['# Simulation Time [s]'] 
            conflicts_dict['UAS1'] += [UAS1]
            conflicts_dict['UAS2'] += [UAS2]
            conflicts_dict['distance'] += [distance]
            conflicts_dict['time'] += [time]




    cpalog_filtered = pd.DataFrame.from_dict(conflicts_dict)
    return cpalog_filtered

def filterCPADeliveryPoint(conflog_filtered, statelog, threshold = 0.25):
    df = pd.DataFrame()
    for idx, row in conflog_filtered.iterrows():
        if ('DELIVERY' in row['UAS1']) or ('DELIVERY' in row['UAS2']):
            time = row['time']        
            try:
                uas1 = statelog[(statelog['# simt'] == time) & (statelog[' id'] == row['UAS1'] ) ].iloc[0]
                lat1 = uas1[' lat']
                lon1 = uas1[' lon']
            except:
                print(time)
                print(statelog[ (statelog[' id'] == row['UAS1'] ) ])
                lat1 = 0
                lon1 = 0
                
            try:
                uas2 = statelog[(statelog['# simt'] == time) & (statelog[' id'] == row['UAS2'] ) ].iloc[0]
                lat2 = uas2[' lat']
                lon2 = uas2[' lon']
            except:
                print(time)
                print(statelog[ (statelog[' id'] == row['UAS2'] ) ])
                lat2 = 0
                lon2 = 0

            lat_ref = 52.3594
            lon_ref = 9.7499

            d1 = distnace(lat_ref,lon_ref,lat1,lon1)
            d2 = distnace(lat_ref,lon_ref,lat2,lon2)

            if (d1 < threshold) or (d2 < threshold):
                delivery_region = True
            else:
                delivery_region = False
                df = pd.concat([df, pd.DataFrame(row).T]).reset_index(drop=True)
            
    return df

# Batch analysis
### All the experimentes are analised at the same time. First we have to define all the folders where the logs files are stored. The final output is a a DataFrame with all the metrics

In [ ]:
##### Definded by user ######

# We create the paths to all the folders where the logs are stored
use_case = 3

if use_case == 1:
    # EXERCISE 1 #
    densities = ['low', 'medium']
    exp = ['0','1','2', '3', '4', '5', '6', '7', '8', '9', '0_no_wind', '1_no_wind', '2_no_wind']
    mode = ['ref', 'D2C2']
    folder_path = r"G:\Unidades compartidas\04_PROYECTOS I+D+i\2021 USEPE\iii) Project\WP5 Simulation and Validation\00 Last mile delivery use case\03. BlueSky outputs\{}_density\exp_{}\{}"
    code = [0,1,2]
    threshold_distance = 0.1 # distance to filter the conflicts near the delivery point
elif use_case == 3:
    # EXERCISE 3 # 
    densities = ['low', 'medium']
    exp = ['1','2', '3', '4', '5', '6', '7', '8', '9', '10']
    mode = ['ref', 'd2c2']
    folder_path = r"/home/ror/ws/scenarios/exercise_3/output/{}_{}_{}/reduced_capacity"
    code = [2,0,1]
    threshold_distance = 0
else: 
    # Improvements #
    densities = ['improvements']
    exp = ['0','1','2', '3', '4', '5', '6', '7', '8', '9']
    mode = []
    folder_path = r"G:\Unidades compartidas\04_PROYECTOS I+D+i\2021 USEPE\iii) Project\WP5 Simulation and Validation\00 Last mile delivery use case\03. BlueSky outputs\{}\{}"
    code = [0,1,2]

exercises = [(x,y,z) for x in densities for y in exp for z in mode]

folder_list = []
for exe in exercises:
    a = exe[0]
    b = exe[1]
    c = exe[2]

    folder = folder_path.format(exe[code[0]],exe[code[1]],exe[code[2]])
    folder_list += [folder]

In [ ]:
results = {}
results['Density'] = []
results['Dataset'] = []
results['Scenario'] = []
results['Started Flights'] = []
results['Completed Flights'] = []
results['Avg. Distance Flown'] = []
results['Avg. flying time'] = []
results['Conflicts'] = []
results['Avg. conflict time'] = []
results['LoS'] = []
results['NMACs'] = []
if use_case == 1:
    results['Conflicts (without delivery point)'] = []
    results['Avg. conflict time (without delivery point)'] = []
    results['LoS (without delivery point)'] = []
    results['NMACs (without delivery point)'] = []

for folder, exe in zip(folder_list, exercises):
    
    a = exe[0]
    b = exe[1]
    c = exe[2]
    
    print(' ')
    print(folder)
    try:
        files_dict = readFolder(folder)
    except FileNotFoundError:
        continue
    
    print(' ')
    print([a,b,c])
        
    if not files_dict:
        continue
        
    conflog = files_dict['USEPECONFLOG']
    cpalog = files_dict['USEPECPALOG']
    flightlog = files_dict['USEPEFLIGHTLOG']
    loslog = files_dict['USEPELOSLOG']
    statelog = files_dict['USEPESTATELOG']
    
    started_flights, completed_flights, avg_distance_flown, avg_flight_time = analyseFlightLog(flightlog)
    
    conflicts, avg_time_in_conflict, conflog_filtered = analyseConflicts(conflog, threshold=20)
    
    conflog_filtered_delivery = filterDeliveryPoint(conflog_filtered, statelog, threshold = threshold_distance)
    
    los, _ , loslog_filtered = analyseLoS(loslog, threshold=20)
    
    loslog_filtered_delivery = filterDeliveryPoint(loslog_filtered, statelog, threshold = threshold_distance)
    
    NMAC, NMAC_df = analyseCpa(cpalog, threshold=20)
    
    NMAC_df_delivery = filterCPADeliveryPoint(NMAC_df, statelog, threshold = threshold_distance)
    
    results['Density'] += [a]
    results['Dataset'] += [b]
    results['Scenario'] += [c]
    results['Started Flights'] += [started_flights]
    results['Completed Flights'] += [completed_flights]
    results['Avg. Distance Flown'] += [avg_distance_flown]
    results['Avg. flying time'] += [avg_flight_time]
    results['Conflicts'] += [conflicts]
    results['Avg. conflict time'] += [avg_time_in_conflict]
    
    
    results['LoS'] += [los]
    
    
    results['NMACs'] += [NMAC]
    
    if use_case == 1:
        results['Conflicts (without delivery point)'] += [len(conflog_filtered_delivery)]
        results['Avg. conflict time (without delivery point)'] += [conflog_filtered_delivery['time_in_conflict'].mean()]
        results['LoS (without delivery point)'] += [len(loslog_filtered_delivery)]
        results['NMACs (without delivery point)'] += [len(NMAC_df_delivery)]
    
df_result = pd.DataFrame.from_dict(results)

   

In [ ]:
df_result.sort_values(by=['Density', 'Scenario', 'Dataset'])

# Additional analysis

In [ ]:
##### Definded by user ######
folder_ref = r"/home/ror/ws/scenarios/exercise_3/output/ref_medium_1/reduced_capacity"
folder_sol = r"/home/ror/ws/scenarios/exercise_3/output/d2c2_medium_1/reduced_capacity"
#############################

In [ ]:
statelog_sol, conflog_sol, loslog_sol, cpalog_sol, flightlog_sol = readLogs(folder_sol)
statelog_ref, conflog_ref, loslog_ref, cpalog_ref, flightlog_ref = readLogs(folder_ref)

statelog_sol = statelog_sol[['SURVEILLANCE_' not in x for x in statelog_sol[' id']]]
statelog_ref = statelog_ref[['SURVEILLANCE_' not in x for x in statelog_ref[' id']]]

In [ ]:
# Total time spent by drones at each altitude. 
ax1 = statelog_ref[' alt'].plot.hist()
ax1.set_xlabel('altitude [m]')
ax1.set_title('Total time spent by drones at each altitude - REFERENCE')
plt.pyplot.show()

ax2  = statelog_sol[' alt'].plot.hist()
ax2.set_xlabel('altitude [m]')
ax2.set_title('Total time spent by drones at each altitude - D2C2')
plt.pyplot.show()

In [ ]:
# Number of CPAs at each altitude

def plotAltitudeCPA(statelog, cpalog, mode):
    
    cpalog = cpalog[cpalog['# Simulation Time [s]'].astype(float) <=3600]
    lat = []
    lon = []
    alt = []
    for idx, row in cpalog.groupby([' UAS1', ' UAS2'], as_index=False)[' Distance [m]'].min().iterrows():
        
        drone1 = row[' UAS1']
        drone2 = row[' UAS2']
        cpa = row[' Distance [m]']

        time = float(cpalog[(cpalog[' UAS1'] == drone1) & (cpalog[' Distance [m]'] == cpa)]['# Simulation Time [s]'].iloc[0])
        lat += [ statelog[(statelog['# simt'] == time) & (statelog[' id'] == drone1)][' lat'].iloc[0] ]
        lon += [ statelog[(statelog['# simt'] == time) & (statelog[' id'] == drone1)][' lon'].iloc[0] ]
        alt += [ statelog[(statelog['# simt'] == time) & (statelog[' id'] == drone1)][' alt'].iloc[0] ]

    ax1 = pd.Series(alt).plot.hist()
    ax1.set_xlabel('altitude [m]')
    ax1.set_title('Number of CPAs at each altitude - {}'.format(mode))
    plt.pyplot.show()
    
plotAltitudeCPA(statelog_ref, cpalog_ref, 'REFERENCE')
plotAltitudeCPA(statelog_sol, cpalog_sol, 'D2C2')